#### 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import warnings
from tqdm import tqdm

In [ ]:
# 경고 메시지 무시
warnings.filterwarnings(action='ignore', category=UserWarning, module='xgboost')

In [ ]:
data = pd.read_csv('machine_learing_data/머신러닝_통합.csv')
data.head(5)

In [ ]:
# 필요한 열 제외
data = data.drop(columns=['curArtist', 'curTitle'])

# 결측치 처리
data = data.fillna('')

# 범주형 변수를 category 타입으로 변환
categorical_cols = ['curColor', 'curColor2', 'curColor3', 'curObject', 'curHead', 'curExplain', 'genre', 'song']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# 'song' 열을 Label Encoding
label_encoder = LabelEncoder()
data['song'] = label_encoder.fit_transform(data['song'])

# 데이터 분리
X = data.drop(columns=['song'])
y = data['song']

# 학습용 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DMatrix 생성
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

#### XGBoost 모델의 특징
- 효율적인 구현: XGBoost는 gradient boosting 알고리즘을 효율적으로 구현한 라이브러리입니다. 이를 통해 빠른 학습 속도와 높은 예측 성능을 달성할 수 있습니다.
- 다양한 작업 지원: XGBoost는 분류, 회귀, 순위 예측 등 다양한 기계학습 작업을 지원합니다.
- 자동 특성 중요도 계산: XGBoost는 자동으로 각 특성의 중요도를 계산하여 제공합니다. 이를 통해 특성 선택 및 모델 해석에 활용할 수 있습니다.
- 병렬 처리 지원: XGBoost는 병렬 처리를 지원하여 대용량 데이터에 대한 학습이 가능합니다.
#### XGBoost 모델의 하이퍼파라미터
- max_depth: 트리의 최대 깊이를 제한하는 하이퍼파라미터입니다. 과적합을 방지하기 위해 적절한 값을 선택해야 합니다.
- learning_rate: 각 트리에서 업데이트되는 가중치의 크기를 조절하는 하이퍼파라미터입니다. 작은 값을 사용하면 과적합을 방지할 수 있습니다.
- n_estimators: 생성할 트리의 개수를 지정하는 하이퍼파라미터입니다. 적절한 값을 선택하여 모델의 성능을 향상시킬 수 있습니다.
- min_child_weight: 리프 노드에서 필요한 최소 샘플 수를 지정하는 하이퍼파라미터입니다. 과적합을 방지하는 데 도움이 됩니다.

In [ ]:
# 모델 파라미터 설정
params = {
    'objective': 'multi:softmax',
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_class': len(label_encoder.classes_),
    'enable_categorical': True
}

# 학습 시간 측정
start_time = time.time()
model = xgb.train(params, dtrain, num_boost_round=100)
end_time = time.time()

# 학습 시간
training_time = end_time - start_time
print(f'학습 시간: {training_time} 초')

In [ ]:
# 모델 평가
y_pred = model.predict(dtest)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
# 예측된 레이블을 원래 값으로 변환
y_pred_original = label_encoder.inverse_transform(y_pred.astype(int))
print(y_pred_original)